In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.cross_validation import train_test_split
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
np.random.seed(1000)

import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ros_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'

if ros_path in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [19]:
# Getting the data: oxford flowers data
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data(one_hot=True)
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33)

In [20]:
# Creating the model class
class AlexNet:
    @staticmethod
    def build():
        model = Sequential()
        
        # First layer conv2d
        model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        
        # Second conv layer
        
        model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        
        # Third conv layer
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # Fourth conv layer
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        # Fifth conv layer
        model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        
        # Dense layer
        model.add(Flatten())
        model.add(Dense(4096, input_shape=(224*224*3,)))
        model.add(Activation('relu'))
        model.add(Dropout(0.4))
        model.add(BatchNormalization())
        
        # Second dense layer
        model.add(Dense(4096))
        model.add(Activation('relu'))
        model.add(Dropout(0.4))
        model.add(BatchNormalization())
        
        # Third dense layer
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.4))
        model.add(BatchNormalization())
        
        # Output layer
        model.add(Dense(17))
        model.add(Activation('softmax'))
        model.summary()
        return model

In [21]:
model = AlexNet.build()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.fit(x, y, batch_size=64, epochs=5, verbose=1,validation_split=0.2, shuffle=True)

Train on 1088 samples, validate on 272 samples
Epoch 1/5
1088/1088 [==============================] - 71s 65ms/step - loss: 2.2628 - acc: 0.3869 - val_loss: 8.3921 - val_acc: 0.2574
Epoch 2/5
1088/1088 [==============================] - 66s 61ms/step - loss: 1.8631 - acc: 0.4265 - val_loss: 4.2857 - val_acc: 0.3603
Epoch 3/5
1088/1088 [==============================] - 67s 61ms/step - loss: 1.7152 - acc: 0.4835 - val_loss: 3.4834 - val_acc: 0.4191
Epoch 4/5
1088/1088 [==============================] - 66s 60ms/step - loss: 1.5986 - acc: 0.5129 - val_loss: 2.8770 - val_acc: 0.4118
Epoch 5/5
1088/1088 [==============================] - 63s 58ms/step - loss: 1.3934 - acc: 0.5735 - val_loss: 3.3196 - val_acc: 0.4301


In [72]:
model.save_weights("weights", overwrite=True)

In [23]:
model.load_weights("weights")

In [8]:
probs = model.predict(x[np.newaxis,0])
prediction = probs.argmax(axis=1)

In [27]:
for i in np.random.choice(np.arange(0, len(y_test)), size=(17,)):
    # classify the digit
    probs = model.predict(X_test[np.newaxis, i])
    prediction = probs.argmax(axis=1)
 
    # show the image and prediction
    print("[INFO] Predicted: {}, Actual: {}".format(prediction[0], np.argmax(y_test[i])))

[INFO] Predicted: 2, Actual: 1
[INFO] Predicted: 14, Actual: 5
[INFO] Predicted: 6, Actual: 3
[INFO] Predicted: 2, Actual: 7
[INFO] Predicted: 15, Actual: 15
[INFO] Predicted: 5, Actual: 5
[INFO] Predicted: 13, Actual: 13
[INFO] Predicted: 9, Actual: 7
[INFO] Predicted: 15, Actual: 15
[INFO] Predicted: 3, Actual: 3
[INFO] Predicted: 1, Actual: 10
[INFO] Predicted: 12, Actual: 12
[INFO] Predicted: 13, Actual: 13
[INFO] Predicted: 15, Actual: 9
[INFO] Predicted: 0, Actual: 0
[INFO] Predicted: 6, Actual: 4
[INFO] Predicted: 11, Actual: 12
